# Neuropixels Visual Coding Tutorial

## Bernstein Workshop 2020: Leveraging Open Datasets from the Allen Brain Observatory for Computational Neuroscience

The Allen Institute has publicly released data from *in vivo* extracellular electrophysiology ("ecephys") experiments in awake mice viewing a variety of visual stimuli. This tutorial will teach you how to access and analyze this data using the AllenSDK, an open-source Python library. It assumes you have basic knowledge of Python and some familiarity with the types of data generated by electrophysiological recordings. 

This tutorial is divided into 5 parts:
* <a href='#Part-1:-Data-access'>Data access</a>
* <a href='#Part-2:-Reading-data-from-an-NWB-file'>Reading data from an NWB file</a>
* <a href='#Part-3:-Aligning-data-to-stimulus-presentations'>Aligning data to stimulus presentations</a>
* <a href='#Part-4:-Receptive-field-mapping'>Receptive field mapping</a>
* <a href='#Part-5:-Decoding-Natural-Images'>Decoding natural images</a>

# Part 1: Data access

The data from Allen Brain Observatory experiments are organized into *sessions*, where each session is a distinct period of continuous recording. During a Neuropixels session we collect:

- spike times and other spike features (such as mean waveforms) from up to 6 Neuropixels probes inserted simultaneously into a mouse brain
- local field potentials from each probe
- behavioral data, such as running speed and eye position
- the timing and identity of visual stimuli which were presented to the mouse
- cell-type specific optogenetic stimuli applied at the end of the session

The AllenSDK contains code for accessing across-session (project-level) metadata as well as code for accessing detailed within-session data. The standard workflow is to use project-level tools, such as `EcephysProjectCache` to identify and access sessions of interest, then delve into those sessions' data using `EcephysSession`.

If you're running analysis locally, the **`EcephysProjectCache` object of the AllenSDK** is the easiest way to retrieve this data. This object abstracts away the details of on-disk file storage, and delivers the data to you as ready-to-analyze Python objects. The cache will automatically keep track of which files are stored locally, and will download additional files on an as-needed basis.

In this tutorial, we'll use an `EcephysProjectCache` object to access the data via the **Amazon Web Services (AWS)** Allen Brain Observatory Simple Storage Service (S3) bucket. This is an AWS Public Dataset located at `arn:aws:s3:::allen-brain-observatory` in region `us-west-2`. Launching a Jupyter notebook instance on AWS will allow you to access the complete dataset without having to download anything locally. This includes around 80 TB of raw data files, which are not accessible via the AllenSDK.

For a more detailed looked at data access, check out [this tutorial](https://allensdk.readthedocs.io/en/latest/_static/examples/nb/ecephys_data_access.html).

## Using the AllenSDK to retrieve data

To get started with this approach, first take care of the necessary imports:

In [ ]:
import os
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

Before downloading the data, you must decide where the **`manifest.json`** file lives. This file serves as the map that guides the `EcephysProjectCache` object to any files that are stored locally, and uses a network connection to retrieve any files that haven't been downloaded yet.

When you initialize a local cache for the first time, it will create the manifest file at the path that you specify. This file lives in the same directory as the rest of the data, so make sure you put it somewhere that has plenty of space available. If you want to download the complete dataset (58 experiments), you'll need 855 GB of space, split across the following files:

1. CSV files containing information about sessions, probes, channels and units (58.1 MB)
2. NWB files containing spike times, behavior data, and stimulus information for each session (146.5 GB total, min file size = 1.7 GB, max file size = 3.3 GB)
3. NWB files containing LFP data for each probe (707 GB total, min file size = 0.9 GB, max file size = 2.7 GB)

When you need to access the data in subsequent analysis sessions, you should point the `EcephysProjectCache` object to an _existing_ `manifest.json` file; otherwise, it will try to re-download the data in a new location.

Since we're using AWS for this demonstration, we'll point to the existing manifest on the Allen Institute's S3 bucket: 

In [ ]:
data_directory = '/mnt/nvme0/ecephys_cache_dir_2' # UPDATE ME!!!

manifest_path = os.path.join(data_directory, "manifest.json")

Now we can create the cache object, specifying both the local storage directory (the `manifest_path`) and the remote storage location (the Allen Institute data warehouse).

This will prepare the cache to download four files:

1. `sessions.csv` (7.8 kB)
2. `probes.csv` (27.0 kB)
3. `channels.csv` (6.6 MB)
4. `units.csv` (51.4 MB)

Each one contains a table of information related to its file name. If you're using the AllenSDK, you won't have to worry about how these files are formatted. Instead, you'll load the relevant data using specific accessor functions: `get_session_table()`, `get_probes()`, `get_channels()`, and `get_units()`. These functions return a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html?highlight=dataframe) containing a row for each item and a column for each metric.

Let's take a closer look at what's in the `sessions.csv` file:

The `sessions` DataFrame provides a high-level overview of the Neuropixels Visual Coding dataset. The index column is a unique ID, which serves as a key for accessing the physiology data for each session. The other columns contain information about:

- the session type (i.e., which stimulus set was shown?)
- the age, sex, and genotype of the mouse (in this dataset, there's only one session per mouse)
- the number of probes, channels, and units for each session
- the brain structures recorded (CCFv3 acronyms)

If we want to find all of recordings from male Sst-Cre mice that viewed the Brain Observatory 1.1 stimulus and contain units from area LM, we can use the following query:

The `filtered_sessions` table contains the three sessions that meet these criteria.

The code above uses standard syntax for filtering pandas DataFrames. If this is unfamiliar to you, we strongly recommend reading through the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/). The AllenSDK makes heavy use of pandas objects, so we don't have to come up with our own functions for working with tabular data.

Let's take a look at another DataFrame, extracted from the `probes.csv` file.

The `probes` DataFrame contains information about the Neuropixels probes used across all recordings. Each row represents one probe from one recording session, even though the physical probes may have been used in multiple sessions. Some of the important columns are:

- `ecephys_session_id`: the index column of the `sessions` table
- `sampling_rate`: the sampling rate (in Hz) for this probe's spike band; note that each probe has a unique sampling rate around 30 kHz. The small variations in sampling rate across probes can add up to large offsets over time, so it's critical to take these differences into account. However, all of the data you will interact with has been pre-aligned to a common clock, so this value is included only for reference purposes.
- `lfp_sampling_rate`: the sampling rate (in Hz) for this probe's LFP band NWB files, after 2x downsampling from the original rate of 2.5 kHz
- `name`: the probe name is assigned based on the location of the probe on the recording rig. This is useful to keep in mind because probes with the same name are always targeted to the same cortical region and enter the brain from the same angle (`probeA` = AM, `probeB` = PM, `probeC` = V1, `probeD` = LM, `probeE` = AL, `probeF` = RL). However, the targeting is not always accurate, so the actual recorded region may be different.
- `phase`: the data may have been generated by one of two "phases" of Neuropixels probes. **3a** = prototype version; **PXI** = publicly available version ("Neuropixels 1.0"). The two phases should be equivalent from the perspective of data analysis, but there may be differences in the noise characteristics between the two acquisition systems.
- `channel_count`: the number of channels with spikes or LFP data (maximum = 384)

The `channels.csv` file contains information about each of these channels.

The most important columns in the `channels` DataFrame concern each channel's location in physical space. Each channel is associated with a location along the probe shank (`probe_horizontal_position` and `probe_vertical_position`), and may be linked to a coordinate in the Allen Common Coordinate framework (if CCF registration is available for that probe).

The information about channel location will be merged into the `units` DataFrame, which is loaded from `units.csv`:

This DataFrame contains metadata about the available units across all sessions. By default, the AllenSDK applies some filters to this table and only returns units above a particular quality threshold.

The default filter values are as follows:

- `isi_violations` < 0.5
- `amplitude_cutoff` < 0.1
- `presence_ratio` > 0.95

For more information about these quality metrics and how to interpret them, please refer to [this tutorial](https://allensdk.readthedocs.io/en/latest/_static/examples/nb/ecephys_quality_metrics.html).

If you want to see _all_ of the available units, it's straightfoward to disable the quality metrics filters when retrieving this table: 

As you can see, the number of units has increased substantially, but some fraction of these units will be incomplete or highly contaminated. Understanding the meaning of these metrics is a critical part of analyzing the Neuropixels dataset, so we strongly recommend learning how to interpret them correctly.

Let's look at the location of all of these units within the CCF, to get a sense of the areas that this dataset covers:

In [ ]:
structure_ids = units.ecephys_structure_id
valid_structure_ids = np.invert(np.isnan(structure_ids))
all_structure_ids, labels = np.unique(units.ecephys_structure_id[valid_structure_ids], return_inverse=True)
lr_coords = units.left_right_ccf_coordinate[valid_structure_ids]
dv_coords = units.dorsal_ventral_ccf_coordinate[valid_structure_ids]

plt.figure(figsize=(10,10))
plt.scatter(lr_coords, -dv_coords, s=1, c=labels, cmap='rainbow')
plt.xlim([6000,11000])
plt.ylim([-5000,0])
plt.xlabel('Left/Right CCF Coordinate')
plt.ylabel('Dorsal/Ventral CCF Coordinate')
plt.grid('on')

plt.text(8400,-1300, 'cortex', fontsize=16)
plt.text(7800,-2300, 'hippocampus', fontsize=16)
plt.text(7000,-3200, 'thalamus', fontsize=16)

The above plot shows the location of all units that have been registered to the CCF, viewed in a coronal section looking from the front of the brain. You can see stripes for the cortex, hippocampus, and thalamus.

# Part 2: Reading data from an NWB file

We package each session's data into a Neurodata Without Borders 2.0 (NWB) file. Calling `get_session_data` on your `EcephysProjectCache` will download such a file and return an `EcephysSession` object.

`EcephysSession` objects contain methods and properties that access the data within an ecephys NWB file and cache it in memory.

Let's look at a file from one of the sessions we selected earlier. If you're accessing the data on your local machine, it may take some time to download the full dataset (approximately 2.6 GB):

## Reading in spike times

First, we'll learn how to explore the available units in this dataset, and look at their spike times and waveforms. 

Information about the recorded units is available in the `session.units` DataFrame:

This table contains a variety of helpful metrics for each unit, such as its firing rate, waveform_amplitude, and CCF structure (`ecephys_structure_acronym`). There are also sorting quality metrics, such as `isi_violations`, `amplitude_cutoff`, and `presence_ratio`. Let's begin by selecting all the units in V1 (`VISp`) that have a low ISI violations score (an indication of low contamination levels):

We will use the index column of this table (labeled `unit_id`) to extract the data for individual units, such as their spike times and waveforms. 

To retrieve the spike times for one unit as a `numpy` array, simply call `session.spike_times[unit_id]`:

Depending on the session you're analyzing, you may see a warning about invalid time intervals. The default session (`756029989`) does have some invalid times, which can result from interruptions in data acquisition or problems with the stimulus display. We've chosen to release sessions with some invalid intervals, in order to avoid throwing away valuable data. To see the intervals (and reasons for invalidation), just type `session.invalid_times`:

Here, we see that Probe E is missing data between 7675 and 7692 seconds. If you're analyzing this section of the data, this will explain why there's a gap during this interval, which is not due to the fact that all units suddenly became silent on this probe.

## Reading in unit waveforms

We store precomputed mean waveforms for each unit in the `mean_waveforms` attribute on the `EcephysSession` object. This is a dictionary which maps unit ids to xarray [DataArrays](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html). These have `channel` and `time` (seconds, aligned to the detected event times) dimensions. The data values are in microvolts, as measured at the recording site.

In [ ]:
units_of_interest = V1_units.index.values

waveforms = {uid: session.mean_waveforms[uid] for uid in units_of_interest}
peak_channels = {uid: session.units.loc[uid, 'peak_channel_id'] for uid in units_of_interest}

# plot the mean waveform on each unit's peak channel/
plot_mean_waveforms(waveforms, units_of_interest, peak_channels)
plt.show()

This plot only includes the peak channel waveform for each unit, but the NWB file contains the waveform across the entire probe, allowing you to analyze how it propagates along the probe axis.

## Stimulus presentation times

Next, we'll look at how to access information about the visual stimuli that were presented.

`session.stimulus_names` will show the stimuli that are available in this session:

Access the stimulus table for a particular stimulus type can be done with the following line of code:

This shows us the parameters for each stimulus presentation (trial), indexed by a unique stimulus presentation ID.

## Behavioral data

We can obtain the mouse's running speed at each point during the experiment by using the `running_speed` attribute of the `session` object. This returns a pandas `DataFrame` whose rows are intervals of time (defined by `start_time` and `end_time` columns), and whose `velocity` column contains mean running speeds within those intervals.

Here we'll plot the running speed trace for an arbitrary chunk of time.

To obtain information about where the animal was looking on the screen, we can call `session.get_screen_gaze_data()`

This table also contains information about the pupil area, which we can plot over time:

## LFP data

LFP, which stands for "local field potential," contains information about low-frequency (0.1-500 Hz) voltage fluctations around each recording site. It's complementary to the spiking activity, and can be analyzed on its own or in conjunction with spikes.

LFP data is accessed on a probe-by-probe basis. Because the LFP data files are so large, they must be downloaded separately from the session NWBs.

Let's take a look at the probes that are available for this session:

Let's look at the structures each of these probes passes through:

In [ ]:
{session.probes.loc[probe_id].description : 
     list(session.channels[session.channels.probe_id == probe_id].ecephys_structure_acronym.unique())
     for probe_id in session.probes.index.values}

Let's choose one and load its associated LFP data:

If you haven't tried to access this data previously, you'll have to wait while the LFP NWB file downloads. Even if you already have the data stored locally, it may still take a minute to load, since the LFP data is quite large.

Once the data is loaded, we can take a closer look at the `lfp` object:

The LFP data is stored as an [xarray.DataArray](http://xarray.pydata.org/en/stable/) object, with coordinates of `time` and `channel`. The xarray library simplifies the process of working with N-dimensional data arrays, by keeping track of the meaning of each axis. If this is your first time encountering xarrays, we strongly recommend reading through the [documentation](http://xarray.pydata.org/en/stable/quick-overview.html). Getting used to xarrays can be frustrating, especially when they don't behave like typical numpy arrays. But they are designed to prevent common mistakes when analyzing multidimensional arrays, so they are well worth learning more about. Plus, the syntax is modeled on that of the [pandas](https://pandas.pydata.org/) library, so if you're familiar with that you already have a head start.

The print-out above already tells us a lot about what the `lfp` object contains. It stores an array with around 12 million points along the `time` axis and 95 points along the `channel` axis. The `time` axis ranges from 13.5 to around 9600 seconds, while the `channel` axis ranges from 850258492 to 850259244 (these are the unique IDs for each channel).

Let's use the `DataArray.sel()` method to select a slice through this array between 100 and 101 seconds:

We see that this new DataArray is smaller than before; it contains the same number of channels, but only 1250 samples, due to the LFP sample rate of ~1250 Hz.

Let's plot the data for one of the channels:

Alternatively, we can visualize this slice  of data using matplotlib's `imshow` method:

Note that we've transposed the original array to place the time dimension along the x-axis. We've also configured the plot so that the origin of the array is in the lower-left, so that that channels closer to the probe tip are lower in the image.

A few things to note about this plot:

* The units of the LFP are volts, so the color scale ranges from -1 to +1 mV
* Even though there are 384 channels on the Neuropixels probe, there are only 95 channels in this plot. That's because only every 4th channel is included in the NWB file (resulting in 40 micron vertical spacing). In addition, the reference channels and channels far outside the brain have been removed.
* The top of the plot is relatively flat. This corresponds to channels that are outside the brain. The LFP channels are originally referenced to a ground wire embedded in the ACSF/agarose mixture about cortex. Before NWB packaging, the LFP data is digitally referenced to the channels outside the brain, to remove noise that's shared across the whole probe.
* There's a large increase in LFP power toward the middle of the probe, which corresponds to channels in hippocampus.

Now it's time to do some more interesting analysis by aligning spikes and LFP to stimulus times!

## Part 3: Aligning data to stimulus presentations

Reading in the data is an essential first step, but most analyses require some form of stimulus alignment. This part of the tutorial will show you how to do that!

First we need to select some trials to use. Let's use full-field flashes (gray-to-black or gray-to-white):

### Aligning spike times

The AllenSDK includes some convenient functions for aligning spikes; if you just want to see all the spike times that fall between the start and end of a set of trials, you can call `session.presentationwise_spike_times`. This takes a list of stimulus presentation IDs and unit IDs, and returns a DataFrame:

The `times` DataFrame contains columns for the stimulus presentation ID and unit ID, as well as the `time_since_stimulus_presentation_onset`, which is the spike time shifted to account for the trial start time.

Using this DataFrame as input, we can make a raster plot for one trial:

### Binning spike times

Instead of returning the individual spike times, we can also use the AllenSDK to bin spike times within a pre-defined interval. This is done by calling the `session.presentationwise_spike_counts` method. Similarly to `session.presentationwise_spike_times`, this method takes stimulus presentation IDs and unit IDs as input. But it also requires one additional argument, which is the edges of the time bins (relative to the start of each trial) for which we want to count spikesL

This has returned an `xarray.DataArray` with the following helpful features:

- Dimensions : Each axis on each data variable is associated with a named dimension. This lets us see unambiguously what the axes of our array mean.
- Coordinates : Arrays of labels for each sample on each dimension.

xarray is nice because it forces code to be explicit about dimensions and coordinates, improving readability and avoiding bugs. However, you can always convert to numpy or pandas data structures as follows:
- to pandas: `spike_counts_ds.to_dataframe()` produces a multiindexed dataframe
- to numpy: `spike_counts_ds.values` gives you access to the underlying numpy array

We can now plot spike counts for a particular presentation:

In [ ]:
presentation_id = presentation_ids[0] # chosen arbitrarily
plot_spike_counts(
    spike_counts.loc[{'stimulus_presentation_id': presentation_id}], 
    spike_counts['time_relative_to_stimulus_onset'],
    'spike count', 
    f'unitwise spike counts on presentation {presentation_id}'
)
plt.show()

We can also average across all presentations, adding a new data array to the dataset. Notice that this one no longer has a `stimulus_presentation_id` dimension, as we have collapsed it by averaging.

... and plot the mean spike counts:

In [ ]:
plot_spike_counts(
    mean_spike_counts, 
    mean_spike_counts['time_relative_to_stimulus_onset'],
    'mean spike count', 
    'mean spike counts on flash presentations'
)
plt.show()

### Aligning LFP data

The AllenSDK does not yet include methods for automatically aligning LFP data to stimulus start times, but we can instead do this by taking advantage of the fact that the data is stored as an xarray:

In [ ]:
trial_window = np.arange(-0.5, 0.5, 1/500)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp = ds['aligned_lfp']

`aligned_lfp` is a DataArray with dimensions of channels x trials x time. It's been downsampled to 500 Hz by changing the time step in the `trial_window` variable.

Because we're using xarrays, the alignment operation is fast, and doesn't require any `for` loops! There's a lot going on here, so we recommend referring to the pandas and xarray documentation if anything is confusing.

Now we can visualize the average LFP, aligned to the trial onset:

In [ ]:
plt.figure(figsize=(8,6))
im = plt.imshow(aligned_lfp.mean(dim='presentation_id'), aspect='auto', origin='lower', vmin=-1e-4, vmax=1e-4)
_ = plt.colorbar(im, fraction=0.036, pad=0.04)
_ = plt.xlabel('Sample number')
_ = plt.ylabel('Channel index')

Here we see the effect of a 250 ms flash stimulus on the LFP. There are two large responses in cortex (channel index 60-80), one corresponding to the stimulus onset (around sample 280), and one corresponding to the stimulus offset (around sample 400).

Note that the voltage range is an order of magnitude less than it was for the single-trial LFP, around -100 to +100 microvolts.

You can use the code sample above to align the LFP to any type of event (e.g. spike times, running onset, optogenetic stimuli) just by changing the `trial_window` and `time_selection` variables.

## Part 4: Receptive field mapping

Because receptive field analysis is so central to interpreting results related from the visual system, every experiment in the Neuropixels Visual Coding dataset includes a standardized receptive field mapping stimulus. This stimulus is always shown at the beginning of the session, and uses the same parameters for every mouse. 

We can look at the `stimulus_presentations` DataFrame in order to examine the parameters of the receptive field mapping stimulus in more detail. The receptive field mapping stimulus consists of drifting Gabor patches with a circular mask, so we're going to filter the DataFrame based on `stimulus_name == 'gabors'`:

There are 3645 trials for the receptive field mapping stimulus. What combination of stimulus parameters is used across these trials? Let's see which parameters actually vary for this stimulus:

We can ignore the parameters related to stimulus timing (`start_time`, `stop_time`, and `duration`), as well as `stimulus_condition_id`, which is used to find presentations with the same parameters. So we're left with `orientation`, `x_position`, and `y_position`.

In [ ]:
print('Unique orientations : ' + str(list(np.sort(rf_stim_table.orientation.unique()))))
print('Unique x positions : ' + str(list(np.sort(rf_stim_table.x_position.unique()))))
print('Unique y positions : ' + str(list(np.sort(rf_stim_table.y_position.unique()))))

What about the drifting grating parameters that don't vary, such as size (in degrees), spatial frequency (in cycles/degree), temporal frequency (in Hz), and contrast?

In [ ]:
print('Spatial frequency: ' + str(rf_stim_table.spatial_frequency.unique()[0]))
print('Temporal frequency: ' + str(rf_stim_table.temporal_frequency.unique()[0]))
print('Size: ' + str(rf_stim_table['size'].unique()[0]))
print('Contrast: ' + str(rf_stim_table['contrast'].unique()[0]))

In order to visualize receptive fields, we're going to use a function in the `ReceptiveFieldMapping` class, one of the stimulus-specific analysis classes in the AllenSDK. Let's import it and create a `rf_mapping` object based on the `session` we loaded earlier:

In [ ]:
from allensdk.brain_observatory.ecephys.stimulus_analysis.receptive_field_mapping import ReceptiveFieldMapping

rf_mapping = ReceptiveFieldMapping(session)

Let's use the same V1 units we extracted previously.

Calculating the receptive field is as simple as calling `get_receptive_field()` with a unit ID as the input argument.

This method creates a 2D histogram of spike counts at all 81 possible stimulus locations, and outputs it as a 9 x 9 matrix. It's summing over all orientations, so each pixel contains the spike count across 45 trials.
 
To plot it, just display it as an image. The matrix is already in the correct orientation so that it matches the layout of the screen (e.g., the upper right pixel contains the spike count when the Gabor patch was in the upper right of the screen).

This particular unit has a receptive field that's near the center of the screen, but shifted slightly to the lower right.

Let's plot the receptive fields for the units in V1 we selected previously:

In [ ]:
def plot_rf(unit_id, index):
    RF = rf_mapping.get_receptive_field(unit_id)
    _ = plt.subplot(6,10,index+1)
    _ = plt.imshow(RF)
    _ = plt.axis('off')
    
_ = plt.figure(figsize=(10,6))
_ = [plot_rf(RF, index) for index, RF in enumerate(V1_units.index.values)]

It's important to look at the receptive fields for any the visual cortex units you're analyzing, especially if you're looking at interactions between units in different areas. The strength cross-area interactions will vary depending on the degree of receptive field overlap of the recording sites.

## Part 5: Decoding Natural Images

In the last part of this tutorial, we'll use the data to answer an interesting scientific question: can we use activity in primary visual cortex to decode the identity of the image the mouse is currently viewing?

To do this, we'll first extract the stimulus table for the `natural_scenes` presentations:

Then, we can align the units in V1 to those presentation times, using the `presentationwise_spike_times` method we learned about earlier:

The input to our classification algorithm will be an *N* presentations x *M* units matrix containing the total number of spikes for each presentation/unit combination.

We can create this using the pandas `groupby` and `pivot_table` methods:

In [ ]:
spikes["count"] = np.zeros(spikes.shape[0])
spikes = spikes.groupby(["stimulus_presentation_id", "unit_id"]).count()

design = pd.pivot_table(
    spikes, 
    values="count", 
    index="stimulus_presentation_id", 
    columns="unit_id", 
    fill_value=0.0,
    aggfunc=np.sum
)

design

...and the target values we want the classifier to learn will be the numeric identifiers of the images that were presented on each trial:

In [ ]:
targets = scene_presentations.loc[design.index.values, "frame"]
targets

We'll use scikit-learn to do the prediction:

In [ ]:
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

The following code trains a support vector machine to classify images based on a subset of the spike counts. It does this 5 times (using a different randomized subset of the data):

In [ ]:
design_arr = design.values.astype(float)
targets_arr = targets.values.astype(int)

labels = np.unique(targets_arr)

accuracies = []
confusions = []

for train_indices, test_indices in KFold(n_splits=5).split(design_arr):
    
    clf = svm.SVC(gamma="scale", kernel="rbf")
    clf.fit(design_arr[train_indices], targets_arr[train_indices])
    
    test_targets = targets_arr[test_indices]
    test_predictions = clf.predict(design_arr[test_indices])
    
    accuracy = 1 - (np.count_nonzero(test_predictions - test_targets) / test_predictions.size)
    print(f"accuracy: {accuracy}")
    
    accuracies.append(accuracy)
    confusions.append(confusion_matrix(y_true=test_targets, y_pred=test_predictions, labels=labels))
    
print(" ")
print(f"mean accuracy: {np.mean(accuracy)}")
print(f"chance: {1/labels.size}")

The mean accuracy represents the probability that the classifier is able to predict the correct image. Given that there are 119 images to choose from (chance = 0.008), the performance is impressive!

We can now plot the average confusion matrix, which shows the likelihood that the classifier will predict the presence of any of the 119 possible images, given the actual image:

From this, we can see that some images are more readily decodable than others. Let's take a look at the most decodable and least decodable images:

In [ ]:
best = labels[np.argmax(np.diag(mean_confusion))]
worst = labels[np.argmin(np.diag(mean_confusion))]

fig, ax = plt.subplots(1, 2, figsize=(16, 8))

best_image = cache.get_natural_scene_template(best)
ax[0].imshow(best_image, cmap=plt.cm.gray)
ax[0].set_title("most decodable", fontsize=24)

worst_image = cache.get_natural_scene_template(worst)
ax[1].imshow(worst_image, cmap=plt.cm.gray)
ax[1].set_title("least decodable", fontsize=24)

plt.show()

## Conclusion

Congratulations on finishing this tutorial! For additional tutorials and documentation, we recommend reading through the [AllenSDK ReadTheDocs site](https://allensdk.readthedocs.io/en/latest/visual_coding_neuropixels.html). 

It may also be helpful to look at the preprints [describing the Neuropixels dataset]((https://www.biorxiv.org/content/10.1101/805010v1) or [comparing it to the imaging 2P dataset](https://www.biorxiv.org/content/10.1101/2020.08.10.244723v1). This will give you a taste of the type of analysis that's been done with this dataset so far, which  barely scratches the surface of what's possible. This data was made to be shared, and we're incredibly excited to see what others do with it!